In [2]:
# CUDA_VISIBLE_DEVICES=0 vllm serve LGAI-EXAONE/EXAONE-4.0-32B --port 8005 --served-model-name exaone-4-32b
#? 위와 같은 명령어로 GPU 하나에다가 EXAONE 실행함

In [2]:
import mlflow
import dspy
import glob
import os
import pandas as pd
from dspy.evaluate import Evaluate

In [3]:
DATA_DIRECTORY = "/home/eoeldroal/WorkPlace/Legal/Legal-DSPy/Data/LU"

In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("DSPy")
mlflow.dspy.autolog()

In [65]:
lm = dspy.LM('openai/exaone-4-32b', api_key='no', api_base='http://0.0.0.0:8005/v1') # 이렇게 뒷부분에 v1를 명시해줘야 올바른 엔드포인트로 간다. 
o4_mini = dspy.LM('openai/o4-mini', max_tokens=20_000, temperature = 1.0)
dspy.configure(lm = lm, track_usage=True)

In [66]:
# 1) 시그니처로 선언하고, 몇 가지 구성을 전달합니다.
COT = dspy.ChainOfThought('passage, question, options, statements -> answer : int')

In [9]:
def load_prompts_with_indices(file_path: str) -> list:
    """단일 CSV 파일에서 프롬프트를 로드하고, 추적을 위해 행 인덱스를 함께 반환합니다."""
    prompts_with_indices = []
    if os.path.exists(file_path) and file_path.endswith(".csv"):
        df = pd.read_csv(file_path)
        for idx, row in df.iterrows():
            prompt_parts = []
            if pd.notna(row.get("지문")):
                prompt_parts.append(f"[passage]\n{row['지문']}")
            if pd.notna(row.get("질문")):
                prompt_parts.append(f"[question]\n{row['질문']}")
            if pd.notna(row.get("보기")):
                prompt_parts.append(f"[list of statements]\n{row['보기']}")
            if pd.notna(row.get("선택지")):
                prompt_parts.append(f"[option]\n{row['선택지']}")
            if pd.notna(row.get("label")):
                prompt_parts.append(f"[label]\n{row['label']}")

            prompt = "\n\n".join(prompt_parts)
            if prompt:
                prompts_with_indices.append((idx, prompt))
    return prompts_with_indices

In [10]:
csv_files = glob.glob(os.path.join(DATA_DIRECTORY, "*.csv"))
list_prompts_with_indices = []
for csv_file in csv_files :
    prompts_with_indices = load_prompts_with_indices(csv_file)
    list_prompts_with_indices.append(prompts_with_indices)

In [11]:
import re

def extract_year_from_path(file_path):
    """파일 경로에서 연도를 추출합니다."""
    year_pattern = r'(20[0-9]{2})'
    match = re.search(year_pattern, file_path)
    return int(match.group(1)) if match else None

def create_year_based_dataset(csv_files, list_prompts_with_indices):
    """
    CSV 파일들을 연도별로 분리하여 데이터셋을 생성합니다.
    2024년 -> 훈련용, 2025년 -> 테스트용
    """
    train_data = []  # 2024년 데이터
    test_data = []   # 2025년 데이터
    
    for file_idx, (csv_file, prompts_with_indices) in enumerate(zip(csv_files, list_prompts_with_indices)):
        year = extract_year_from_path(csv_file)
        print(f"파일: {csv_file} -> 연도: {year}")
        
        if year == 2024:
            train_data.extend(prompts_with_indices)
        elif year == 2025:
            test_data.extend(prompts_with_indices)
        else:
            print(f"경고: 알 수 없는 연도 {year} (파일: {csv_file})")
    
    print(f"\n2024년 데이터 (훈련용): {len(train_data)}개")
    print(f"2025년 데이터 (테스트용): {len(test_data)}개")
    
    return train_data, test_data

# 연도별 데이터 분리
train_prompts, test_prompts = create_year_based_dataset(csv_files, list_prompts_with_indices)

파일: /home/eoeldroal/WorkPlace/Legal/Legal-DSPy/Data/LU/언어이해_2024.csv -> 연도: 2024
파일: /home/eoeldroal/WorkPlace/Legal/Legal-DSPy/Data/LU/언어이해_2025.csv -> 연도: 2025

2024년 데이터 (훈련용): 30개
2025년 데이터 (테스트용): 30개


In [37]:
def create_dspy_examples_from_prompts(prompts_with_indices):
    """
    프롬프트 데이터를 DSPy Example 객체로 변환합니다.
    
    Args:
        prompts_with_indices: (인덱스, 프롬프트) 튜플들의 리스트
    
    Returns:
        list: DSPy Example 객체들의 리스트
    """
    examples = []
    
    for idx, prompt in prompts_with_indices:
        # 프롬프트를 섹션별로 파싱
        sections = {}
        current_section = None
        current_content = []
        
        for line in prompt.split('\n'):
            if line.startswith('[') and line.endswith(']'):
                # 이전 섹션 저장
                if current_section:
                    sections[current_section] = '\n'.join(current_content).strip()
                # 새 섹션 시작
                current_section = line[1:-1]  # [] 제거
                current_content = []
            elif current_section:
                current_content.append(line)
        
        # 마지막 섹션 저장
        if current_section:
            sections[current_section] = '\n'.join(current_content).strip()
        
        # DSPy Example 생성
        example_data = {
            'passage': sections.get('passage', ''),
            'question': sections.get('question', ''),
            'options': sections.get('option', ''),
            'statements': sections.get('list of statements', ''),
            'answer': sections.get('label', ''),
            'original_index': idx
        }
        
        # DSPy Example 생성 - passage, question, options, statements를 입력으로 설정
        input_fields = ['passage', 'question']
        if 'options' in example_data:
            input_fields.append('options')
        if 'statements' in example_data:
            input_fields.append('statements')
            
        example = dspy.Example(**example_data).with_inputs(*input_fields)
        examples.append(example)
    
    return examples

# 연도별 DSPy 데이터셋 생성
train_set = create_dspy_examples_from_prompts(train_prompts)
test_set = create_dspy_examples_from_prompts(test_prompts)

print(f"훈련 데이터셋 (2024년): {len(train_set)}개")
print(f"테스트 데이터셋 (2025년): {len(test_set)}개")

print("\n훈련 데이터 첫 번째 예시:")
if train_set:
    example = train_set[0]
    print(f"입력 필드: {example.inputs()}")
    print(f"정답: {example.answer if 'answer' in example else 'N/A'}")

print("\n테스트 데이터 첫 번째 예시:")
if test_set:
    example = test_set[0]
    print(f"입력 필드: {example.inputs()}")
    print(f"정답: {example.answer if 'answer' in example else 'N/A'}")

훈련 데이터셋 (2024년): 30개
테스트 데이터셋 (2025년): 30개

훈련 데이터 첫 번째 예시:
입력 필드: Example({'passage': "규범교의적 학문을 자처하는 법학은 학문성에 관한 논쟁에 시달려 왔다. 입법자의 권력 행사로 법전의 한마디가 바뀌면, 오랫동안 가꾼 해석의 축적이 순식간에 무용지물이 되기 때문이다. 이에 대한 도전으로서 알베르트는 경험적 반증가능성을 강조하는 비판적 합리주의에 입각하여 법학의 학문성을 새롭게 이해하고자 한다. \n알베르트는 우선 법학의 은폐된 특징을 신학과의 비교를 통해 문제 삼는다. 법학은 당국의 고시에서 진리를 얻어내는 점에서 신학과 구조적 유사성을 가지기 때문이다. 신학이 경전의 해석을 통해 권위를 확보하듯, 법학은 법전을 확인하고 문제 해결과 관련하여 이를 해석한다. 이때 경전이나 법전은 학문적 비판이나 성찰의 대상이 아니라 해석적 권위의 원천이자 근거가 될 따름이다. 그가 보기에 법학이 신학과의 구조적 유사성을 탈피하려면, 해석에서 자연법이냐 사회학이나의 양자택일을 감수해야 한다. 선택의 결과는 자명하다. 절대성을 가진 규범적 현실에 의해 실정법이 구성되고 또 구속된다고 보는 견해는 신적인 힘으로 설립된 세계를 믿는 관점에 의해서만 유지될 수 있기 때문이다. 알베르트는 법을 인간의 문화적 성취로 간주하고, 사회적 삶의 사실 중 사회 구성원의 상호 행위 조종의 영역에 속하는 것으로 본다. \n물론 이 경우에도 법을 현실주의적으로 보느냐, 규범주의적으로 보느냐의 문제는 남는다. 알베르트는 법을 사회적 사실로, 법학을 경험과학으로 볼 것을 주장한다. 그에 따르면 규범에 관한 법학적 언명은 규범 자체와 다르게 규범성이 없으며, 이 구별을 무시한다면 규범의 인식적 파악이라는 이념은 사라지게 된다. 그는 법률 문언의 규범성은 인정하지만, 그 문언에 관하여 의미를 밝히는 법학은 다르다고 말한다. \n법학에 대한 알베르트의 현실주의적 파악에는 곤란해 보이는 점도 있다. 예컨대, 법률 문언에 흠결이 존재하여 적극적

In [53]:
sample_example = test_set[0]  # 첫 번째 테스트 데이터 사용

In [ ]:
# Chain of Thought 시그니처 정의 - 법률 문서 질의응답용
class LegalQA(dspy.Signature):
    """주어진 질문에 대해 단계별 추론을 통해 정확한 답을 선택합니다."""
    passage = dspy.InputField(desc="법률 관련 지문")
    question = dspy.InputField(desc="지문에 대한 질문") 
    options = dspy.InputField(desc="선택지")
    statements = dspy.InputField(desc="보기")
    answer : int = dspy.OutputField(desc="최종 답 (숫자)")

In [74]:
# Chain of Thought 모듈 생성
cot_legal_qa = dspy.ChainOfThought(LegalQA)

print("Chain of Thought 모듈이 생성되었습니다.")
print(f"시그니처: {LegalQA}")

Chain of Thought 모듈이 생성되었습니다.
시그니처: LegalQA(passage, question, options, statements -> answer
    instructions='법률 문서를 읽고 주어진 질문에 대해 단계별 추론을 통해 정확한 답을 선택합니다.'
    passage = Field(annotation=str required=True json_schema_extra={'desc': '법률 관련 지문', '__dspy_field_type': 'input', 'prefix': 'Passage:'})
    question = Field(annotation=str required=True json_schema_extra={'desc': '지문에 대한 질문', '__dspy_field_type': 'input', 'prefix': 'Question:'})
    options = Field(annotation=str required=True json_schema_extra={'desc': '선택지', '__dspy_field_type': 'input', 'prefix': 'Options:'})
    statements = Field(annotation=str required=True json_schema_extra={'desc': '보기', '__dspy_field_type': 'input', 'prefix': 'Statements:'})
    answer = Field(annotation=int required=True json_schema_extra={'desc': '최종 답 (숫자)', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)


In [75]:
# 샘플 데이터로 Chain of Thought 테스트 실행
if test_set:
    print("=== Chain of Thought 테스트 실행 ===")
    
    # 입력 준비
    inputs = {
        'passage': sample_example.passage,
        'question': sample_example.question,
        'options': sample_example.options,
        'statements': sample_example.statements
    }
    
    print(f"입력 준비 완료")
    
    # MLflow 실행과 함께 추론 수행
    with mlflow.start_run(run_name="cot_single_test"):
        try:
            # Chain of Thought 실행
            result = cot_legal_qa(**inputs)
            
            print("\\n=== 결과 ===")
            print(f"추론 과정:\\n{result.reasoning}")
            print(f"\\n모델 답: {result.answer}")
            print(f"정답: {sample_example.answer}")
            print(f"정답 일치: {str(result.answer).strip() == str(sample_example.answer).strip()}")
            
        except Exception as e:
            print(f"오류 발생: {e}")
            import traceback
            traceback.print_exc()
else:
    print("테스트할 데이터가 없습니다.")

=== Chain of Thought 테스트 실행 ===
입력 준비 완료
\n=== 결과 ===
추론 과정:\n1. 지문에서 형법 개혁 운동과 범죄소설 열풍의 관계를 설명하며, 형법 개혁의 원동력이 되었다고 언급되어 있으므로 ①은 일치한다.
2. 지문에서 뉴게이트 소설은 당대의 지배적 범죄 담론에 대한 대항 담론을 선전했다고 언급되어 있다. 따라서 범죄를 질병으로, 형벌을 치료로 이해한 담론을 강화했다는 ②는 지문 내용과 일치하지 않는다.
3. 지문에서 캐서린에 대한 독자들의 반응이 작가의 의도와 다르게 나타났다는 내용이 있으므로 ③은 일치한다.
4. 지문에서 뉴게이트 소설의 대중적 인기와 기득권층의 우려가 언급되어 있으므로 ④는 일치한다.
5. 지문에서 올리버 트위스트는 범죄 유혹을 물리쳤다는 내용이 있으므로 ⑤는 일치한다.
\n모델 답: 2
정답: 2
정답 일치: True
🏃 View run cot_single_test at: http://127.0.0.1:5000/#/experiments/366534923737239846/runs/42f9b0c343c94ae799a704de9c2487d2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/366534923737239846


Trace(trace_id=tr-4fe6c066be26b798133e422ac3cc51c1)

In [76]:
# 추가 테스트: 다른 예시들도 몇 개 더 테스트해보기
print("=== 추가 테스트 (처음 3개 예시) ===")

for i in range(min(3, len(test_set))):
    example = test_set[i]
    print(f"\\n--- 테스트 {i+1} ---")
    print(f"질문: {example.question[:100]}...")
    
    inputs = {
        'passage': example.passage,
        'question': example.question,
        'options': example.options if hasattr(example, 'options') else "선택지 없음"
    }
    
    try:
        result = cot_legal_qa(**inputs)
        print(f"모델 답: {result.answer}")
        print(f"정답: {example.answer}")
        print(f"일치: {'✓' if str(result.answer).strip() == str(example.answer).strip() else '✗'}")
    except Exception as e:
        print(f"오류: {e}")
        
print("\\n=== 테스트 완료 ===")

2025/08/08 19:21:17 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['passage', 'question', 'options']. Missing: ['statements'].
2025/08/08 19:21:17 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['passage', 'question', 'options']. Missing: ['statements'].
2025/08/08 19:21:17 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['passage', 'question', 'options']. Missing: ['statements'].


=== 추가 테스트 (처음 3개 예시) ===
\n--- 테스트 1 ---
질문: 윗글의 내용과 일치하지 않는 것은?...
모델 답: 2
정답: 2
일치: ✓
\n--- 테스트 2 ---
질문: ㉠∼㉤에 대한 이해로 적절하지 않은 것은?...
모델 답: 4
정답: 2
일치: ✗
\n--- 테스트 3 ---
질문: 윗글에서 추론한 것으로 가장 적절한 것은?...
모델 답: 5
정답: 5
일치: ✓
\n=== 테스트 완료 ===


[Trace(trace_id=tr-f9c3b5df701917a3070d0b2fa48cfcf8), Trace(trace_id=tr-133a7a28dc5c89ff518301008ab872a1), Trace(trace_id=tr-c0bd94e36bd5737382d85eb5e5d240ad)]

In [77]:
def validate_answer(example, pred, trace=None):
    return int(example.answer) == int(pred.answer)

In [78]:
# 코드에서 재사용할 수 있는 평가기 설정
evaluator = Evaluate(devset=test_set, num_threads=5, display_progress=True, display_table=10)
with mlflow.start_run(run_name="Cot_before_optimization"):
    # 평가 실행
    evaluator(cot_legal_qa, validate_answer)

Average Metric: 9.00 / 11 (81.8%):  37%|███▋      | 11/30 [00:44<01:07,  3.55s/it]

2025/08/08 19:22:13 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 17.00 / 30 (56.7%): 100%|██████████| 30/30 [02:03<00:00,  4.12s/it]

2025/08/08 19:23:32 INFO dspy.evaluate.evaluate: Average Metric: 17 / 30 (56.7%)


,passage,question,options,statements,example_answer,original_index,reasoning,pred_answer,validate_answer
0,문학이 사회와 그 구성원의 삶을 반영한다는 명제는 법의 영역에도 적용된다. 문학적 서사는 한 시대의 법인식과 정의관을 ...,윗글의 내용과 일치하지 않는 것은?,"①형법 개혁 운동은 범죄소설 열풍의 계기이자 성과였다. ②뉴게이트 소설은 범죄를 질병으로, 형벌을 치료로 이해한 당대 ...",,2,0,"1. 지문에서 형법 개혁 운동과 범죄소설 열풍의 관계를 설명하며, 형법 개혁의 원동력이 되었다고 언급되어 있으므로 ①은...",2,✔️ [True]
1,문학이 사회와 그 구성원의 삶을 반영한다는 명제는 법의 영역에도 적용된다. 문학적 서사는 한 시대의 법인식과 정의관을 ...,㉠∼㉤에 대한 이해로 적절하지 않은 것은?,①㉠은 죄에 비해 과한 형을 구형하거나 사형 선고를 남발하는 현상을 가리킨다. ②㉡은 살아남기 위해 주어진 계급적 위치...,,2,1,"1. ㉠ ""피에 굶주린 형법전""은 수감자를 타락하게 만들고 단순 절도범마저 공동체 복귀 기회를 박탈하는 과도한 형벌을 ...",4,
2,문학이 사회와 그 구성원의 삶을 반영한다는 명제는 법의 영역에도 적용된다. 문학적 서사는 한 시대의 법인식과 정의관을 ...,윗글에서 추론한 것으로 가장 적절한 것은?,①디킨스는 법의 부조리에 대한 비판과 범죄의 해악에 대한 훈계를 한 작품에서 동시에 수행할 수 없다고 보았을 것이다. ...,,5,2,"1. 지문은 뉴게이트 소설과 같은 문학 작품이 법과 사회에 미치는 영향을 설명하고 있다. 특히, 불워-리턴, 에인즈워스...",5,✔️ [True]
3,동서양의 전설에 나오는 귀신 중 흡혈귀는 문학의 소재로 오래 활용되었다. 특히 흡혈귀는 슬라브 또는 헝가리의 전설에 자...,윗글의 내용과 일치하지 않는 것은?,①코프로포르피리노젠I은 포르피린의 한 종류이다. ②미오글로빈과 시토크롬은 헴을 보철그룹으로 가지고 있는 단백질이다. ③...,,4,3,"1. **① 코프로포르피리노젠I은 포르피린의 한 종류이다.** - 지문에서 ""코프로포르피리노젠I""은 포르피린 합성 중간...",4,✔️ [True]
4,동서양의 전설에 나오는 귀신 중 흡혈귀는 문학의 소재로 오래 활용되었다. 특히 흡혈귀는 슬라브 또는 헝가리의 전설에 자...,윗글에서 추론한 내용으로 가장 적절한 것은?,①콕스는 조지 3세의 모발에서 비소 대사와 관련된 효소 유전자의 결함을 찾고자 하였을 것이다. ②조지 3세는 불면증과 ...,,2,4,"1. **옵션 ① 분석**: 콕스는 조지 3세의 모발에서 비소를 발견했지만, 헴 합성과 관련된 유전자 결함을 찾으려 했...",2,✔️ [True]
5,동서양의 전설에 나오는 귀신 중 흡혈귀는 문학의 소재로 오래 활용되었다. 특히 흡혈귀는 슬라브 또는 헝가리의 전설에 자...,"<보기>를 바탕으로 본문을 이해할 때, 가장 적절한 것은?","①효소 ㉠, ㉡의 산물은 도스포르피린증 환자의 체내에 축적될 것이다. ②효소 ㉢의 산물이 코프로포르피리노젠I로 전환되는...",<보 기> 헴 합성은 (가)와 같은 다단계 효소 촉매 과정에 의하여 일어난다. 효소는 ‘기질’의 화학적 구조를 변화시키...,3,5,"1. 보기의 표를 분석하여 각 효소와 그에 해당하는 포르피린증을 확인한다. 2. ①의 경우, 효소 ㉠과 ㉡의 산물은 각...",2,
6,"기존의 역사가들이 민주주의, 노예제와 같은 정치․사회제도의 모델을 찾기 위해 고대 그리스와 로마에 주목했다면, 최근에는...",윗글에 대한 이해로 적절하지 않은 것은?,①플라톤은 파이데라스티아의 대상을 일정한 지적 성장 단계의 소년으로 한정했다. ②크세노폰은 에라스테스를 소년의 육체를 ...,,2,6,"1. **①**은 지문에서 플라톤이 ""소년을 사랑하는 사람들은 아무 소년이나 사랑하는 것이 아니라 이성을 갖기 시작한 ...",2,✔️ [True]
7,"기존의 역사가들이 민주주의, 노예제와 같은 정치․사회제도의 모델을 찾기 위해 고대 그리스와 로마에 주목했다면, 최근에는...",윗글로 보아 다음 설명 중 가장 적절한 것은?,①아테네와 스파르타에서는 모두 이십 대 청년이 에로메노스에서 배제되었다. ②아테네에서와 달리 스파르타에서의 에라스테스는...,,3,7,"1. 아테네와 스파르타의 소년애 관행 비교: - 아테네: 에로메노스(소년)의 나이는 17세 이하로 제한되었고, 에라스테...",3,✔️ [True]
8,"기존의 역사가들이 민주주의, 노예제와 같은 정치․사회제도의 모델을 찾기 위해 고대 그리스와 로마에 주목했다면, 최근에는...","윗글과 <보기>를 연결하여 평가할 때, 가장 적절한 것은?",①도자기에 그려진 장면은 에로메노스와 에라스테스 관계에 대한 골든의 해석과 상충하는군. ②그리스 도자기의 소년애 장면은...,<보 기> 고대 그리스 도자기에 묘사된 소년애 장면은 이성애 장면에 비해 훨씬 덜 노골적이다. 이는 자유민 소년이 성적...,2,8,"1. **도자기 장면과 골든의 해석**: 보기에서 도자기 장면은 소년애가 덜 노골적이며, 그리스 사회가 자유민 소년을 ...",2,✔️ [True]
9,사법심사는 다수주의의 예외로 간주되기도 한다. 민주적 절차로 선출된 의회나 행정부의 결정이 합헌 여부를 기준으로 무효화...,네 가지 모델에 대한 설명으로 가장 적절한 것은?,①긍정적 반응 모델은 연방대법원의 전문성과 공정성에 대한 대중의 불신을 반영한다. ②연방대법원의 결정을 여론이 즉각 수...,,3,9,"주어진 지문은 사법심사의 정치적 독립성과 여론의 영향에 대한 네 가지 모델(긍정적 반응 모델, 반발 모델, 양극화 모델...",3,✔️ [True]


🏃 View run Cot_before_optimization at: http://127.0.0.1:5000/#/experiments/366534923737239846/runs/74ecdf5252a849d68fe507cd2b980447
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/366534923737239846


[Trace(trace_id=tr-70f1e9d4f3c686bbd38020c753ea3fd2), Trace(trace_id=tr-34ee79ec366524be83587f722e967553), Trace(trace_id=tr-30f5481e3e25182e30f2a718d103c3f5), Trace(trace_id=tr-067aad7a58fafaf5ff544a4e1319fbb3), Trace(trace_id=tr-57f81289593d934e052bd7bfe515b41e), Trace(trace_id=tr-f8056708ec1233cef76f413171848873), Trace(trace_id=tr-0071424f473fbcc2b51500197bffeacd), Trace(trace_id=tr-1f0b562da74d3e024d5d7d412b574fd9), Trace(trace_id=tr-133b6e0fbdcaf0aaed58af099489591c), Trace(trace_id=tr-82528625504ee3ea2f0d032a1fec0b1f)]

```
[[ ## reasoning ## ]]`, then `[[ ## answer ## ]]` (must be formatted as a valid Python int), and then ending with the marker for `[[ ## completed ## ]]`.[|endofturn|]\n[|assistant|]\n<think>\n\n</think>\n\n....
```
위처럼, thinking을 하지 않도록 구성되어 있다. 

- 괴제 1. 엑사원의 thinking과 compatible하도록 adapter 등을 재구성하자.
- 과제 2. 또한 자동 프롬프트 구성기도 재구성하자.
- 과제 3. Chat complete api도 검토해 보자. 

정리하자면, thinking을 활성화해 주는 모듈과 시그니처를 재구성해야 하는 것이다. 
현재 형식은 한계가 많다. 

In [79]:
THREADS = 24

kwargs = dict(num_threads=THREADS, teacher_settings=dict(lm=o4_mini), prompt_model=lm)
optimizer = dspy.MIPROv2(metric=validate_answer, auto="medium", **kwargs)

kwargs = dict(requires_permission_to_run=False, max_bootstrapped_demos=4, max_labeled_demos=4)
optimized_module = optimizer.compile(cot_legal_qa, trainset=train_set, **kwargs)

2025/08/08 19:25:02 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 18
minibatch: False
num_fewshot_candidates: 12
num_instruct_candidates: 6
valset size: 24

2025/08/08 19:25:02 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/08/08 19:25:02 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/08/08 19:25:02 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...


Bootstrapping set 1/12
Bootstrapping set 2/12
Bootstrapping set 3/12


 17%|█▋        | 1/6 [00:30<02:30, 30.08s/it]/home/eoeldroal/miniconda3/envs/DSPy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 6/6 [02:08<00:00, 21.36s/it]


Bootstrapped 3 full traces after 5 examples for up to 1 rounds, amounting to 6 attempts.
Bootstrapping set 4/12


 67%|██████▋   | 4/6 [01:07<00:36, 18.42s/it]2025/08/08 19:29:12 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/08 19:29:57 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example Example({'passage': '금융, 마케팅, 의료 등 다양한 분야에서 생성되는 빅데이터는 많은 경우 개인정보를 포함하고 있어 데이터를 활용하는 과정에서 민감한 개인정보가 유출될 가능성이 있다. 따라서 빅데이터 구축 과정에서 개인정보의 전부 또는 일부를 삭제하거나 대체함으로써 개인의 신원이 드러나지 않도록 하면서도 해당 데이터의 활용성을 최대한 유지할 수 있도록 하는 개인정보 비식별화 기술을 사용한다. \n데이터 집합에서 정보를 표현하는 최소 단위를 속성이라고 하고 다양한 속성들의 조합으로 표현된 하나의 정보를 레코드라고 한다. 데이터 집합은 이 레코드들의 집합이다. 비식별화 기술은 속성을 식별자, 준식별자, 일반속성, 민감속성으로 구분한다. 주민번호와 같이 그 자체만으로도 누구인지 식별 가능한 속성이 식별자이다. 반면에 성별, 연령, 주소와 같이 개인에 대한 직접적인 식별은 불가능하지만 이들 속성이 결합하면 개인에 대한 식별이 가능해지는 속성을 준식별자라고 한다. 성별, 이름, 연령으로 구성되어 있는 원본 데이터 집합이 있을 때, 이름에서 성씨만을 남겨 비식별 데이터 집합을 만들었다고 하자. 비록 이름은 성만 남기고 가려져 있지만 \'남성\'이 유일하거나, 성이 \'이씨\'이면서 \'35세인 사람이 유일하다면, 원본에 이 두 사람이 포함된 사실을 알면서 이들 각자의 유일한 속성값 조합을 미리 알고 있는 사람은 특정 개인을 재식별할 수 있다. 일반적으로 개인정보는 개인의 여러 속성과 결합하여 사용된다

...

2025/08/08 19:52:29 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 10.00 / 19 (52.6%):  83%|████████▎ | 20/24 [00:37<00:04,  1.10s/it]

2025/08/08 19:52:42 ERROR dspy.utils.parallelizer: Error for Example({'passage': '투표 참여에 대한 설명은 유권자가 투표에 참여하기 위해 치르는 비용에 주목한다. 예를 들어 투표소가 거주지와 가깝거나 이동하기 쉬운 곳에 있을수록 유권자들이 더 쉽게 투표할 수 있다. 또한 투표 참여 비용의 큰 부분을 차지하는 것이 <가> 선거와 후보에 대한 정보를 획득하고 처리하는 비용 </가>이다. 일반적으로 사회경제적 지위가 높은 유권자들이 그렇지 않은 유권자들에 비해 더 열심히 투표에 참여하는 이유는 전자가 이러한 비용을 더 낮게 체감하기 때문이다. \n선거일 날씨도 투표 참여를 결정하는 데 있어 비용의 구성 요소가 될 수 있다. 비가 오는 날에는 투표소에 가거나 줄을 서서 차례를 기다리는 것이 불편한 일이기 때문이다. 따라서 기존 연구들은 궂은 날씨가 유권자가 투표하러 가는 것을 망설이게 한다는 데 동의한다. 다만 지금까지 학문적 관심의 초점은 궂은 날씨로 인 한 비용 증가가 실제로 투표율을 낮출 만큼 큰 문제인가에 맞춰져 왔다. 어떤 학자들은 날씨가 유발하는 비용 증가는 미미하다고 주장하지만, 다른 학자들은 작은 불편으로 인한 추가 비용도 상당수 유권자 사이에서 투표와 기권의 선택을 뒤바꿀 수 있다고 본다. \n미국 대통령선거를 대상으로 한 최근 연구에 따르면 주 단위에서 강수량과 투표율을 비교했을 때, 강수량이 평년보다 1인치 증가할 때 투표율은 약 2.4% 포인트 감소했다. 다만 이 연구는 <나> 주별 강수량을 측정하기 위해 그 주에서 가장 큰 도시의 선거 당일 강수량을 대리지표(proxy)로 활용 </나> 했다는 점에서 비판의 대상이 되었다. 그러나 이러한 문제를 교정한 다른 연구에서도 강수량의 증가가 투표율 감소를 가져온다는 증거가 제시되었다. \n그런데 투표와 관련된 비용에는 투표에 참여하는 데 필요한 직접비용뿐 아니라, <다> 투표에 참여하느라 다른 선택을 포기하는 데서 오는 기회비용 </다>도

Average Metric: 12.00 / 22 (54.5%): 100%|██████████| 24/24 [00:43<00:00,  1.83s/it]

2025/08/08 19:52:45 INFO dspy.evaluate.evaluate: Average Metric: 12.0 / 24 (50.0%)
2025/08/08 19:52:45 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 50.0 with parameters ['Predictor 0: Instruction 3', 'Predictor 0: Few-Shot Set 11'].
2025/08/08 19:52:45 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [37.5, 45.83, 41.67, 50.0, 45.83, 58.33, 41.67, 50.0, 50.0, 58.33, 58.33, 58.33, 50.0, 58.33, 50.0, 54.17, 50.0, 50.0, 50.0]
2025/08/08 19:52:45 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 58.33
2025/08/08 19:52:45 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/08/08 19:52:45 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 58.33!


In [80]:
optimized_module

predict = Predict(StringSignature(passage, question, options, statements -> reasoning, answer
    instructions='주어진 법률 문서를 분석하고, 질문에 대한 정답을 선택하기 위해 단계별 추론을 수행하세요. 특히, 알베르트와 사비니의 주장을 비교하면서 각 옵션의 적절성을 평가해야 합니다. 정답을 선택할 때는 문서의 내용과 정확히 일치하는지 확인하고, 모순이 있는 부분을 주의 깊게 검토하세요.'
    passage = Field(annotation=str required=True json_schema_extra={'desc': '법률 관련 지문', '__dspy_field_type': 'input', 'prefix': 'Passage:'})
    question = Field(annotation=str required=True json_schema_extra={'desc': '지문에 대한 질문', '__dspy_field_type': 'input', 'prefix': 'Question:'})
    options = Field(annotation=str required=True json_schema_extra={'desc': '선택지', '__dspy_field_type': 'input', 'prefix': 'Options:'})
    statements = Field(annotation=str required=True json_schema_extra={'desc': '보기', '__dspy_field_type': 'input', 'prefix': 'Statements:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_

'주어진 법률 문서를 분석하고, 질문에 대한 정답을 선택하기 위해 단계별 추론을 수행하세요. 특히, 알베르트와 사비니의 주장을 비교하면서 각 옵션의 적절성을 평가해야 합니다. 정답을 선택할 때는 문서의 내용과 정확히 일치하는지 확인하고, 모순이 있는 부분을 주의 깊게 검토하세요.'

미친년아

In [82]:
with mlflow.start_run(run_name="Cot_after_optimization"):
    evaluator(optimized_module, validate_answer)

Average Metric: 20.00 / 30 (66.7%): 100%|██████████| 30/30 [00:00<00:00, 32.39it/s]

2025/08/08 19:55:01 INFO dspy.evaluate.evaluate: Average Metric: 20 / 30 (66.7%)


,passage,question,options,statements,example_answer,original_index,reasoning,pred_answer,validate_answer
0,문학이 사회와 그 구성원의 삶을 반영한다는 명제는 법의 영역에도 적용된다. 문학적 서사는 한 시대의 법인식과 정의관을 ...,윗글의 내용과 일치하지 않는 것은?,"①형법 개혁 운동은 범죄소설 열풍의 계기이자 성과였다. ②뉴게이트 소설은 범죄를 질병으로, 형벌을 치료로 이해한 당대 ...",,2,0,"1) 지문에서 형법 개혁 운동과 범죄소설 열풍의 관계를 명시적으로 언급하며, 개혁의 원동력이 되었다고 설명한다. 2) ...",2,✔️ [True]
1,문학이 사회와 그 구성원의 삶을 반영한다는 명제는 법의 영역에도 적용된다. 문학적 서사는 한 시대의 법인식과 정의관을 ...,㉠∼㉤에 대한 이해로 적절하지 않은 것은?,①㉠은 죄에 비해 과한 형을 구형하거나 사형 선고를 남발하는 현상을 가리킨다. ②㉡은 살아남기 위해 주어진 계급적 위치...,,2,1,"1. ㉠ ""피에 굶주린 형법전""은 과도한 형벌과 사형 선고를 비판하는 표현으로, ①의 설명과 일치한다. 2. ㉡ ""자연...",4,
2,문학이 사회와 그 구성원의 삶을 반영한다는 명제는 법의 영역에도 적용된다. 문학적 서사는 한 시대의 법인식과 정의관을 ...,윗글에서 추론한 것으로 가장 적절한 것은?,①디킨스는 법의 부조리에 대한 비판과 범죄의 해악에 대한 훈계를 한 작품에서 동시에 수행할 수 없다고 보았을 것이다. ...,,5,2,1. 지문에서 디킨스는 법의 부정의를 고발하면서도 해학과 권선징악을 통해 법질서를 교란하지 않는 방식을 선택했다고 언급...,5,✔️ [True]
3,동서양의 전설에 나오는 귀신 중 흡혈귀는 문학의 소재로 오래 활용되었다. 특히 흡혈귀는 슬라브 또는 헝가리의 전설에 자...,윗글의 내용과 일치하지 않는 것은?,①코프로포르피리노젠I은 포르피린의 한 종류이다. ②미오글로빈과 시토크롬은 헴을 보철그룹으로 가지고 있는 단백질이다. ③...,,4,3,1) 지문에서 코프로포르피리노젠I은 포르피린 합성 중간물질로 명시되어 있으므로 ①은 일치한다. 2) 미오글로빈과 시토크...,4,✔️ [True]
4,동서양의 전설에 나오는 귀신 중 흡혈귀는 문학의 소재로 오래 활용되었다. 특히 흡혈귀는 슬라브 또는 헝가리의 전설에 자...,윗글에서 추론한 내용으로 가장 적절한 것은?,①콕스는 조지 3세의 모발에서 비소 대사와 관련된 효소 유전자의 결함을 찾고자 하였을 것이다. ②조지 3세는 불면증과 ...,,2,4,"1) 콕스는 조지 3세의 모발에서 헴 합성과 연관된 유전자 결함을 찾으려 했으나 성공하지 못했고, 대신 비소를 발견했다...",2,✔️ [True]
5,동서양의 전설에 나오는 귀신 중 흡혈귀는 문학의 소재로 오래 활용되었다. 특히 흡혈귀는 슬라브 또는 헝가리의 전설에 자...,"<보기>를 바탕으로 본문을 이해할 때, 가장 적절한 것은?","①효소 ㉠, ㉡의 산물은 도스포르피린증 환자의 체내에 축적될 것이다. ②효소 ㉢의 산물이 코프로포르피리노젠I로 전환되는...",<보 기> 헴 합성은 (가)와 같은 다단계 효소 촉매 과정에 의하여 일어난다. 효소는 ‘기질’의 화학적 구조를 변화시키...,3,5,"1. **①** 도스포르피린증은 효소 ㉠ 결핍으로 발생하며, 기질인 델타아미노레불린산이 축적됩니다. 산물인 포르피린 A...",3,✔️ [True]
6,"기존의 역사가들이 민주주의, 노예제와 같은 정치․사회제도의 모델을 찾기 위해 고대 그리스와 로마에 주목했다면, 최근에는...",윗글에 대한 이해로 적절하지 않은 것은?,①플라톤은 파이데라스티아의 대상을 일정한 지적 성장 단계의 소년으로 한정했다. ②크세노폰은 에라스테스를 소년의 육체를 ...,,2,6,"1) 플라톤은 ""이성을 갖기 시작한 나이의 소년들만을 사랑한다""고 서술하여 파이데라스티아의 대상을 지적 성장 단계의 소...",2,✔️ [True]
7,"기존의 역사가들이 민주주의, 노예제와 같은 정치․사회제도의 모델을 찾기 위해 고대 그리스와 로마에 주목했다면, 최근에는...",윗글로 보아 다음 설명 중 가장 적절한 것은?,①아테네와 스파르타에서는 모두 이십 대 청년이 에로메노스에서 배제되었다. ②아테네에서와 달리 스파르타에서의 에라스테스는...,,3,7,"1. 아테네에서는 17세 이하의 소년이 에로메노스 역할을 했고, 스파르타에서는 30세까지도 이 역할을 유지했다는 점에서...",3,✔️ [True]
8,"기존의 역사가들이 민주주의, 노예제와 같은 정치․사회제도의 모델을 찾기 위해 고대 그리스와 로마에 주목했다면, 최근에는...","윗글과 <보기>를 연결하여 평가할 때, 가장 적절한 것은?",①도자기에 그려진 장면은 에로메노스와 에라스테스 관계에 대한 골든의 해석과 상충하는군. ②그리스 도자기의 소년애 장면은...,<보 기> 고대 그리스 도자기에 묘사된 소년애 장면은 이성애 장면에 비해 훨씬 덜 노골적이다. 이는 자유민 소년이 성적...,2,8,1. 도자기 장면과 골든의 해석: 도자기 장면이 덜 노골적이라는 것은 골든이 주장한 '소년의 명예와 존엄 배려'와 일치...,4,
9,사법심사는 다수주의의 예외로 간주되기도 한다. 민주적 절차로 선출된 의회나 행정부의 결정이 합헌 여부를 기준으로 무효화...,네 가지 모델에 대한 설명으로 가장 적절한 것은?,①긍정적 반응 모델은 연방대법원의 전문성과 공정성에 대한 대중의 불신을 반영한다. ②연방대법원의 결정을 여론이 즉각 수...,,3,9,"1. 긍정적 반응 모델은 연방대법원의 전문성과 공정성에 대한 대중의 신뢰를 전제로 하므로, ①번은 잘못된 설명이다. 2...",3,✔️ [True]


🏃 View run Cot_after_optimization at: http://127.0.0.1:5000/#/experiments/366534923737239846/runs/a31a35fc58834adfb9c1e5f3d7680854
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/366534923737239846


[Trace(trace_id=tr-a0cccf2c7576a778ffc568019ffe2469), Trace(trace_id=tr-99c3d240949997128e8d4e8a02f1a5aa), Trace(trace_id=tr-4c71e9d714ca684573ea218350815f34), Trace(trace_id=tr-f4f02ddf8324221840bee374b2e8a0df), Trace(trace_id=tr-095400196010a9de5dc10941c69597e6), Trace(trace_id=tr-2cbbb6bdcbbe4be9374c4359306fa0c2), Trace(trace_id=tr-5942757dfb707c6bb631af9b2cecdf09), Trace(trace_id=tr-5182fe140bcc9740f66e208b09faa55b), Trace(trace_id=tr-a4b10771492d35df2d2962d008ffbde3), Trace(trace_id=tr-f0ac31bd0006cfcb2c677939fe6c8ab5)]